# Общие выводы по работе

В данном проекте перед нами стояла важная продуктовая задача обнаружения каверов-треков, которая может значительно улучшить качество рекомендаций музыкального сервиса и повысить счастье пользователей Яндекс Музыки. Решение данной задачи может позволить:

- по желанию пользователя можем полностью исключить каверы из рекомендаций;
- показать все каверы на любимый трек пользователя;
- контролировать долю каверов в ленте пользователя.

В ходе решения данной задачи были проведены следующие этапы:
1. Первичный анализ данных;
2. Предообработка данных;
3. Создание моделей, позволяющих решить поставленные задачи.

Рассмотрим их более подробно.

## Первичный анализ данных

В данном разделе мы изучили предоставленные заказчиком данные, обнаружили некоторую специфику данных и выдвинули гипотезы. В частности было установлено:

1. В данных имеются пропуски, например original_track_id указан у относительно малого количества строк. При этом смержить цепочку каверов можно только для 297 строк.
2. В данных с текстом есть дублирования track_id, при разных lyrick_id и иногда даже текстов. Нами было принято решение на препроцессинге выбирать первый трек, а остальные не учитывать. Это сделано для того, чтобы избежать дублирования строк.
3. Текст представлен на разных языках (наиболее часто в данных встречаются треки на английском, значительно реже на испанском, русском и итальянском), и для его векторизации нам потребовалось его перевести.
4. В ходе исследования данных было проведено изучение дополнительных источников, которое позволило установить, что ISRC – это уникальный код, который однозначно и навсегда определяет музыкальную или видео запись, при этом в нём зашифрована полезная информация. Например, ISRC RU — B94 — 13 — 00137 (ISRC RUB941300137) позволяет получить следующую информацию: Код страны (2 буквы)RU = Россия, Код регистранта(3 символа,)B94 = Концертно-продюсерский центр, Год присваивания кода(2 цифры)13 = 2013, Номер записи(5 цифр).
5. Анализируя жанры треков было обнаружено, что в датасете чаще встречаются треки с жанрами FOLK, LATINFOLK и POP дальше с большим отрывом идут ALLROCK, и ROCK.
6. Продолжительность треков имеет распределение близкое к нормальному с длинным правым хвостом. Также имеются треки с 0 значением продолжительности их мы убрали, поскольку в таких треках может быть ошибка.

Также в данном разделе были предложены варианты проведения препроцессинга и дальнейшего создания моделей машинного обучения.

## Предобработка данных

Тетрадь conclusions: Перед тем как осуществить предобработку данных, нам потребовалось перевести все тексты песен, заголовки и определить язык на которых они написаны, если в базе в столбце language стоит None. Для этой цели мы воспользовались API Yandex Translator (настройку подключения производили через YandexCloud).

Сам этап предобработки состоял из 3 подэтапов:
1. Создание объединённого датасета из всех доступных данных и генерация основных признаков;
2. Создание датасета для решения задачи классификации по признаку оригинал / кавер;
3. Создание датасета для задачи группировки каверов и исходного трека.

Рассмотрим, что было сделано на каждом из подэтапов:
1. Создание объединённого датасета из всех доступных данных и генерация основных признаков
    * были написаны функции позволяющие получить подготовленный объединённый датасет;
    * данные функции проводят первичную предобработку текста (переводят всё в нижний регистр, проводят его лемматизацию)
    * были восстановлены пропуски в столбце язык: если в первоначальном датасете отсутсвовал язык, то первоначально мы брали язык текста, если информации всё равно не хватало, то брали язык заголовка;
    * были развёрнуты жанры, выбрано топ 15 признаков по которым провели преобразование типа One Hot Encoding, для остальных признаков жанров создали столбец unknown_genre. Также были созданы столбцы группировки по жанрам, которые можно было получить из названия 'ROCK', 'RAP', 'POP', 'FOLK', 'RUS';
    * из isrc были получены следующие признаки: код страны, год регистрации (при этом сделали его в датасете как категориальным, так и числовым), код регистранта;
    * добавили столбцы явно указывающие на отсутствие той или иной информации.
2. Создание датасета для решения задачи классификации по признаку оригинал / кавер
    * удалили столбцы с разными id;
    * удалили пропуски по целевому признаку;
    * добавили столбец указывающий на очень короткие и очень длинные треки (в EDA было устновлено, что такие треки как правило являются каверами)
    * данные были разделены на трейн и тест (для тестового набора были отдельно получены track_id, поскольку они понадобятся для решения 3 задачи)
3. Создание датасета для задачи получения группировки каверов и исходного трека
    * удалили пропуски в целевом признаке (original_track_id);
    * создали функцию, для получения списка подходящих треков и их количество в датасете на основе разметки;
    * на основании количества треков разделили датасет на тренировочный, тестовый и валидационный (тут разделили на 3 датасета, поскольку модель будет двухуровневая и каждую нужно будет независимо проверить);
    * провели векторизацию и лемматизацию текстов.

## Создание моделей

Тетрадь conclusions: Задача создания модели была разбита на 3 подзадачи:
- классификация трека по признаку кавер или оригинал;
- группировка каверов и исходного трека;
- поиск оригинального трека в цепочке каверов.
Каждую задачу решали отдельно, в результате получили 3 ноутбука с моделингом. Рассмотрим каждую из них отдельно,

### Создание модели классификации трека по признаку кавер / оригинал

Данная задача является задачей бинарной классификации. Для её решения воспользовались заранее подготовленным датасетом. При создании модели прошли следующие этапы:
1. Выбор метрики (учитывая дисбаланс классов выбрали f1 меру)
2. Создание безлайна в виде дамми модели с метрикой 0.1137
3. Обучение выбранной модели (учитывая большое количество категориальных признаков была выбрана модель CatBoostClassifier, которая с дефолтными параметрами показала качество 0.7249
4. Провели анализ важности признаков и подбор гиперпараметров при помощи GridSearchCV и достигли качества модели равное 0.7617, что значительно превосходит базовую модель.

### Создание модели для группировки каверов и исходного трека

Для решения данной задачи было принято решение создать двухуровневую модель:
1. На первом уровне, при помощи FAISS будем искать k ближайших соседей для каждого трека-запроса.
2. На основании списка полученных претендентов строим модель (CatBoostClassifier), которая будет принимать решения являются ли треки каверами друг для друга.

Опишем проделанную работу на каждом из этапов.  
Работая над первым уровнем модели (FAISS) нами было сделано:
- выбрана метрика для анализа модели (Recall@k);
- созданы функции упростившие обучение индекса и получение от него ответа;
- подобраны параметры tfidf, используемые при векторизации текста;
- подобрано оптимальное количество возвращаемых k ближайших.

При работе над вторым уровнем модели:
- выбрана метрика (учитывая дисбаланс также использовали f1 меру);
- рассмотрены разные стратегии формирования датсета;
- обучена Dummy модель в качестве безлайна с метрикой f1 0.1025;
- исследована важность признаков;
- подобраны оптимальные гиперпараметры, которые позволили получить значение метрики на кроссвалидации равное 0.9036
- написана функция, которая позволяет по предобработанному датасету вернуть для каждого трека список его каверов / оригинала в виде строки разделённой пробелом

### Создание модели для поиска исходного трека в цепочке каверов

Решение данной задачи мы решили искать аналитическим путём, а ML-моделью воспользуемся в тех случаях, когда иначе найти решение не удастся. Чтобы найти исходный трек в цепочке каверов была предложена и реализована следующая логика решения задачи:
- сначала проводится анализ цепочки каверов и при отсутствии цепочки как таковой (модель из второй задачи не нашла похожие треки в базе), будем считать сам трек оригиналом;
- если цепочка треков была обнаружена, то получаем самый ранний из треков на основании года, полученному из isrc (по факту год регистрации трека);
- если несколько треков соответствуют самому раннему году, либо если есть треки, для которых год неизвестен, то отдаём их для прогноза модели, обученной для решения задачи классификации кавер / оригинал, и по predict_proba выбираем тот, в котором модель больше уверена.

## Заключительный вывод

В ходе данной работы было предложено решение бизнес-задачи по поиску каверов. Разработанные модели позволяют обнаруживать каверы, объединять треки в кластеры каверов, а также искать исходный трек в цепочке каверов. Это поможет как пользователям продукта (возможность выбора исключительно каверов в своём плейлисте или наоборот прослушивание только оригинала), так и самому продукту, можно проверить качество данных в базе (найти каверы, повторения треков, которые ранее оставались незамеченными).
Если мы умеем с высокой точностью классифицировать каверы и связывать их между собой, то можно предложить пользователю новые возможности для управления потоком треков.

**Рекомендации для бизнеса:**
1. Улучшение качества данных, это позволит получить более высокую метрику на наших моделях, что приведет к повышению лояльности клиента.
2. В дальнейшем рекомендуем добавить в базу данных сведения об исполнителях песен, названиях альбомов, в целях продолжения тестирования гипотез по улучшению качества модели.
3. Продолжить работу над ручной разметкой данных, отдав эту задачу на аутсорсинг.